# Quadratic Variation Completion (QVC)

## Model Description

- **Optimization Problem**

$$\begin{aligned} \min_{\boldsymbol{X}}~&\frac{\gamma}{2}\sum_{n=1}^{N}\boldsymbol{x}_{n}^\top\tilde{\boldsymbol{L}}\boldsymbol{x}_{n} \\ \text{s.t.}~&\|\mathcal{P}_{\Omega}(\boldsymbol{X}-\boldsymbol{Y})\|_{F}\leq\epsilon \end{aligned}$$
where
$$\tilde{\boldsymbol{L}}=\begin{bmatrix}
2 & -1 & 0 & \cdots & 0 & 0 & -1 \\
-1 & 2 & -1 & \cdots & 0 & 0 & 0 \\
0 & -1 & 2 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & -1 & 2 & -1 \\
-1 & 0 & 0 & \cdots & 0 & -1 & 2 \\
\end{bmatrix}$$


- **Augmented  Lagrangian function**

$$\begin{aligned} \mathcal{L}(\boldsymbol{X},\boldsymbol{Z},\boldsymbol{W})=&\frac{\gamma}{2}\sum_{n=1}^{N}\boldsymbol{x}_{n}^{\top}\tilde{\boldsymbol{L}}\boldsymbol{x}_{n}+\frac{\lambda}{2}\|\boldsymbol{X}-\boldsymbol{Z}\|_{F}^{2}+\langle\boldsymbol{W},\boldsymbol{X}-\boldsymbol{Z}\rangle+\frac{\eta}{2}\|\mathcal{P}_{\Omega}(\boldsymbol{Z}-\boldsymbol{Y})\|_{F}^{2} \\ =&\frac{\gamma}{2}\operatorname{tr}(\boldsymbol{X}\tilde{\boldsymbol{L}}\boldsymbol{X}^\top)+\frac{\lambda}{2}\|\boldsymbol{X}-\boldsymbol{Z}\|_{F}^{2}+\langle\boldsymbol{W},\boldsymbol{X}-\boldsymbol{Z}\rangle+\frac{\eta}{2}\|\mathcal{P}_{\Omega}(\boldsymbol{Z}-\boldsymbol{Y})\|_{F}^{2} \end{aligned}$$

- **ADMM Scheme**

With respect to $\boldsymbol{X}$:

$$\begin{aligned} \boldsymbol{X}:=&\arg\,\min_{\boldsymbol{X}}~\frac{\gamma}{2}\operatorname{tr}(\boldsymbol{X}\tilde{\boldsymbol{L}}\boldsymbol{X}^\top)+\frac{\lambda}{2}\|\boldsymbol{X}-\boldsymbol{Z}+\boldsymbol{W}/\lambda\|_{F}^{2} \\ =&(\lambda\boldsymbol{Z}-\boldsymbol{W})(\gamma(\tilde{\boldsymbol{L}}+\tilde{\boldsymbol{L}}^\top)/2+\lambda\boldsymbol{I}_{T})^{-1} \end{aligned}$$

With respect to $\boldsymbol{Z}$:

$$\begin{aligned} \boldsymbol{Z}:=&\arg\,\min_{\boldsymbol{Z}}~\frac{\lambda}{2}\|\boldsymbol{X}-\boldsymbol{Z}-\boldsymbol{W}/\lambda\|_{F}^{2}+\frac{\eta}{2}\|\mathcal{P}_{\Omega}(\boldsymbol{Z}-\boldsymbol{Y})\|_{F}^{2} \\ =&\frac{1}{\lambda+\eta}\mathcal{P}_{\Omega}(\lambda\boldsymbol{X}+\boldsymbol{W}+\eta\boldsymbol{Y})+\mathcal{P}_{\Omega}^{\perp}(\boldsymbol{X}+\boldsymbol{W}/\lambda) \end{aligned}$$

With respect to $\boldsymbol{W}$:

$$\boldsymbol{W}:=\boldsymbol{W}+\lambda(\boldsymbol{X}-\boldsymbol{Z})$$

In [2]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def conv(vec, kernel_size):
    n = vec.shape[0]
    mat = np.zeros((n, kernel_size))
    mat[:, 0] = vec
    for k in range(1, kernel_size):
        mat[:, k] = np.append(vec[n - k :], vec[: n - k], axis = 0)
    return mat

def update_x(z, w, L_tilde, lmbda, gamma):
    x = (lmbda * z - w) @ np.linalg.inv(gamma * (L_tilde + L_tilde.T) / 2 
                                        + lmbda * np.eye(z.shape[1]))
    return x

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def QVC(y_true, y, lmbda, gamma, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    L_tilde = conv(laplacian(T, 1), T)
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = update_x(z, w, L_tilde, lmbda, gamma)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [ ]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3 # 0.5, 0.7, or 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 5e-3 * T
gamma = 5 * lmbda
maxiter = 100
x = QVC(dense_mat, sparse_mat, lmbda, gamma, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>